# Install Directions

In [1]:
# 1. download package
# 2. create app on reddit, MSKrocks!!, dphigrvity2023 https://www.geeksforgeeks.org/scraping-reddit-using-python/
# 3. secret- kR6EWhELqUWmkJHNU0Z4L94tIPyxhA, useragent- GTProject, clientid= heI2hGaX16nfVwvUpN9Ayg
# 4. Scripts\activate.ps1

# questions to answer, over time, themes, related associations

Notes
 time is mentioned today, yesterday, year
 there is moderation
 boyfriend and girlfriend
 referenced age usually young
 reddit limit of 1000

In [103]:
import praw
import pandas as pd
import datetime
import numpy as np 
from tqdm._tqdm_notebook import tqdm_notebook,tnrange,tqdm
import nltk
nltk.download('punkt')
import heapq
import gensim
from gensim import models,corpora
import spacy
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.models.ldamodel import LdaModel
from pprint import pprint
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import Phrases



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Reddit Setup

In [3]:
# Variables
len_posts= 10000
community_name='cancer'
#community_name='asknyc'
len_comment=10

In [4]:
reddit_read_only = praw.Reddit(client_id="heI2hGaX16nfVwvUpN9Ayg",         # your client id
                               client_secret="iydVBDtmgtXXV5xVgHXtPXatC42wdQ",      # your client secret
                               user_agent="GTProject")        # your user agent

subreddit = reddit_read_only.subreddit(community_name)
 
# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 
# Display the title of the Subreddit
print("Title:", subreddit.title)
 
# Display the description of the Subreddit
print("Description:", subreddit.description)

Display Name: cancer
Title: Cancer: Discussion & Support
Description: ######New rules have been implemented

###About Us

This reddit is a place for people with cancer and caregivers to come together and provide support for one other.

###Rules

1. No Blogspam
2. No solicitations
3. No donation requests
4. No "is this cancer" questions
5. No homeopathy / nature / quack medicine
6. Mods have the right to remove your post at their discretion 
7. All posts linking scientific studies must include a summary paragraph written by the poster.

###Misc Info

* Join the /r/Cancer Discord server here: https://discord.gg/cancersupport

* Got an idea to improve /r/Cancer? Make a post describing your thoughts.

* See something thats not supposed to be here? Press the **report button.** And tell us why.

* Need to talk to a mod? [Message us](http://www.reddit.com/message/compose?to=%23cancer).



###External Sites

* [American Cancer Society](http://www.cancer.org/)
* [National Cancer Institute](http

# Top and Hot Posts

In [158]:
top_post = []
for post in subreddit.top(limit=len_posts):
    top_post.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created,post.title+". "+post.selftext])
top_posts = pd.DataFrame(top_post,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created', 'text'])

# hot_post = []
# for post in subreddit.hot(limit=len_posts):
#     hot_post.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created,post.title])
# hot_posts = pd.DataFrame(hot_post,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created','text'])

In [61]:
top_posts.to_csv("top_posts-"+community_name+".csv") 

# Comments

In [7]:
# top_comment = []

# for id in top_posts['id']: 
#     submission = reddit_read_only.submission(id)
#     comments = submission.comments.replace_more(limit=len_comment)
#     comments = submission.comments.list()

#     for comment in comments:
#         comment_id = comment.id
#         create = datetime.datetime.fromtimestamp(comment.created)
#         text = comment.body
#         top_comment.append([id,comment_id,create,text,])
#     df_comments = pd.DataFrame(top_comment,columns=['id','comment_id','created','text'])

# Set up Models

In [159]:
# Define variables and stop words
nlp = spacy.load('en_core_web_sm')
data= top_posts # Define data to use. refer to column text

nlp.Defaults.stop_words |= {"cancer","patient","chemo", "treatment", "chemotherapy", "radiation", "chemo", "yesterday","year","today"
                                ,"day","good", "month", "time","tomorrow", "week", "feel","thing", "think","people"
                                ,"want", "tell", "know","believe","bear", "mountain", "lion","run", "bad", "life","thank", "work"
                                ,"love","come","post","start","live","die","friend","pain","time","update","pass"
                                ,"hour","person","sub","story","happen","guy","fuck","old","morning","need","find","leave"
                                ,"need","help","hope","try","look","way","let","lot","ask","wish","read", "fight","battle","fucking"
                                ,"family","doctor","kill","shit","little","bring","able","great","eat","surgery","stage"}

# Functions
def conv_string(text):
    text_out=[]
    for ind in text:
        a=ind
        a=' '.join(str(v) for v in a)
        text_out.append(a)
    return text_out

def lemma(text):
    text_out=[]
    doc=nlp(text)
    for token in doc:
        text_out.append(token.lemma_)
    return text_out

def clean_up(text):
    removal=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE','AUX']
    text_out = []
    doc= nlp(text)
    for token in doc:
    
        if token.is_stop == False and token.is_alpha and len(token)>=2 and token.pos_ not in removal:
            text_out.append(token)
    return text_out
    



In [160]:
data1= data.text.apply(lambda x:x.lower())
data1=pd.DataFrame({'index':data1.index, 'text':data1.values})

data2 = data1.text.apply(lambda x: clean_up(x)) # returns list in list
data2=pd.DataFrame({'index':data2.index, 'text':data2.values})

data3= conv_string(data2.text)   
data3= pd.DataFrame(data3, columns=['text'])   

data4= data3.text.apply(lambda x: lemma(x))
data4= pd.DataFrame(data4, columns=['text'])  
data4= conv_string(data4.text)   
data4 = pd.DataFrame({'text':data4})

datalist= data4.text.apply(lambda x: clean_up(x)) #cleaned up data
datalist = pd.DataFrame({'text':datalist}) #to dataframe
datalist= conv_string(datalist.text)   # combine to text


# Bag of Words

In [161]:
word2count = {}
for d in datalist:
  
    line = nltk.word_tokenize(d)
    for word in line:
    
        if word not in word2count.keys():
            word2count[word] = 1
            
        else:
            word2count[word] += 1
     
sorted_dict=sorted(word2count.items(), key=lambda x: x[1], reverse=True)

bagofwords=pd.DataFrame(sorted_dict, columns=['word',"freq"])
bagofwords.to_csv("bagofwords-"+community_name+".csv")

# Topic Modelling



## Create matrix

In [162]:
text_tokens = [[text for text in doc.split()] for doc in datalist]

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(text_tokens, min_count=5)
for idx in range(len(text_tokens)):
    for token in bigram[text_tokens[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            text_tokens[idx].append(token)

dictionary = corpora.Dictionary(text_tokens)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_tokens]

# Human readable format of corpus (term-frequency)
# [[(dictionary[id], freq) for id, freq in cp] for cp in doc_term_matrix[:100]]

## Tune Model

In [163]:
# Lda = models.LdaMulticore
# coherenceList_umass = []
# coherenceList_cv = []
# num_topics_list = np.arange(3,26)
# for num_topics in tqdm(num_topics_list):
#     lda= Lda(doc_term_matrix, num_topics=num_topics,id2word = dictionary, 
#              passes=20,chunksize=2000,random_state=3)
#     cm = CoherenceModel(model=lda, corpus=doc_term_matrix, 
#                         dictionary=dictionary, coherence='u_mass')
#     coherenceList_umass.append(cm.get_coherence())
#     cm_cv = CoherenceModel(model=lda, corpus=doc_term_matrix,
#                            texts=datalist, dictionary=dictionary, coherence='c_v')
  

## Plot Coherence Score

In [164]:
# plotData = pd.DataFrame({'Number of topics':num_topics_list,
#                          'CoherenceScore':coherenceList_umass})
# f,ax = plt.subplots(figsize=(10,6))
# sns.set_style("darkgrid")
# sns.pointplot(x='Number of topics',y= 'CoherenceScore',data=plotData)
# plt.axhline(y=-3.9)
# plt.title('Topic coherence')
# plt.savefig('Topic coherence plot.png')

## Model Build

In [165]:
num_topics=5
num_words=3

# 5 topics, 3 words seems to work well, 6 and 3 also

Lda = models.LdaMulticore
lda= Lda(doc_term_matrix, num_topics=num_topics,id2word = dictionary, 
         passes=20,chunksize=2000,random_state=3)

pprint(lda.print_topics())

[(0,
  '0.006*"melanoma" + 0.006*"scan" + 0.005*"oncologist" + 0.005*"free" + '
  '0.005*"result" + 0.005*"tumor" + 0.005*"lung" + 0.004*"remission" + '
  '0.004*"new" + 0.004*"lymph"'),
 (1,
  '0.011*"scan" + 0.009*"tumor" + 0.008*"diagnose" + 0.007*"news" + '
  '0.006*"hospital" + 0.006*"lung" + 0.005*"liver" + 0.005*"happy" + '
  '0.005*"result" + 0.004*"end"'),
 (2,
  '0.006*"dad" + 0.005*"lose" + 0.005*"diagnose" + 0.005*"miss" + 0.004*"care" '
  '+ 0.004*"mean" + 0.004*"end" + 0.004*"hospital" + 0.004*"edit" + '
  '0.003*"stop"'),
 (3,
  '0.014*"mom" + 0.008*"dad" + 0.007*"lose" + 0.006*"hospital" + '
  '0.006*"diagnose" + 0.005*"support" + 0.005*"husband" + 0.004*"care" + '
  '0.004*"talk" + 0.004*"sister"'),
 (4,
  '0.006*"diagnose" + 0.005*"lose" + 0.005*"end" + 0.005*"wife" + 0.005*"stay" '
  '+ 0.004*"school" + 0.004*"hospital" + 0.004*"brain" + 0.004*"body" + '
  '0.004*"hair"')]


In [166]:
# Alternate LDA Model
# lda_model = LdaModel(corpus=doc_term_matrix,
#                    id2word=dictionary,
#                    num_topics=5,
#                    random_state=0,
#                    chunksize=1000,
#                    alpha='auto',
#                    per_word_topics=True)

# pprint(lda_model.print_topics())
# doc_lda = lda_model[doc_term_matrix]

In [167]:
a = lda.show_topics(num_topics,formatted=False,num_words=num_words)
b = lda.top_topics(doc_term_matrix,dictionary=dictionary,topn=10) # This orders the topics in the decreasing order of coherence score

topic2skillb = {}
topic2csb = {}
topic2skilla = {}
topic2csa = {}
num_topics =lda.num_topics
cnt =1

for ws in b:
    wset = set(w[1] for w in ws[0])
    topic2skillb[cnt] = wset
    topic2csb[cnt] = ws[1]
    cnt +=1

for ws in a:
    wset = set(w[0]for w in ws[1])
    topic2skilla[ws[0]+1] = wset
    
for i in range(1,num_topics+1):
    for j in range(1,num_topics+1):  
        if topic2skilla[i].intersection(topic2skillb[j])==topic2skilla[i]:
            topic2csa[i] = topic2csb[j]

finalData = pd.DataFrame([],columns=['Topic','words'])
finalData['Topic']=topic2skilla.keys()
finalData['Topic'] = finalData['Topic'].apply(lambda x: 'Topic'+str(x))
finalData['words']=topic2skilla.values()
finalData['cs'] = topic2csa.values()
finalData.sort_values(by='cs',ascending=False,inplace=True)
finalData.to_csv('CoherenceScore-'+community_name+'.csv')
finalData

,Topic,words,cs
1,Topic2,"{tumor, scan, diagnose}",-1.605936
0,Topic1,"{oncologist, melanoma, scan}",-1.815941
2,Topic3,"{lose, diagnose, dad}",-1.823998
3,Topic4,"{mom, lose, dad}",-1.823998
4,Topic5,"{end, lose, diagnose}",-2.577878


# Visual

In [168]:
def format_topics_sentences(ldamodel=lda, corpus=doc_term_matrix, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=doc_term_matrix, texts=top_posts.text)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.to_excel("dominant-"+community_name+".xls")

c:\Users\sur\VS Code\GTFinalProject\.venv\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.


In [169]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, doc_term_matrix, dictionary,n_jobs=1)
vis
#pyLDAvis.save_html(vis, 'lda'+community_name+'.html')

c:\Users\sur\VS Code\GTFinalProject\.venv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.033646 -0.039277       1        1  36.357284
3     -0.084603 -0.052494       2        1  22.963657
2     -0.036349 -0.014487       3        1  18.905206
4     -0.027564  0.111294       4        1  10.984587
0      0.114869 -0.005036       5        1  10.789265, topic_info=         Term        Freq       Total Category  logprob  loglift
145      scan  217.000000  217.000000  Default  30.0000  30.0000
361       mom  242.000000  242.000000  Default  29.0000  29.0000
246    result  108.000000  108.000000  Default  28.0000  28.0000
352  melanoma   34.000000   34.000000  Default  27.0000  27.0000
214     lymph   61.000000   61.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
139     right   13.174494   77.274729   Topic5  -5.8817   0.4575
361       mom   17.663838  242.691841   Topic5  -5.5884  -0.3937
460      care   14.406484  127.864732   Topic5  -5.7923   0.0433
216      news   14.238223  141.541789   Topic5  -5.8040  -0.0700
50        end   12.177837  176.581333   Topic5  -5.9603  -0.4475

[431 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
958       1  0.342430  advice
958       2  0.162204  advice
958       3  0.144181  advice
958       4  0.144181  advice
958       5  0.216272  advice
...     ...       ...     ...
194       1  0.255638   young
194       2  0.330826   young
194       3  0.135338   young
194       4  0.240601   young
194       5  0.015038   young

[941 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 5, 1])